In [ ]:
!pip install bertopic -qqq
!pip install bertopic[visualization] -qqq
!pip install konlpy -qqq

     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 5.2 MB 42.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 3.8 MB 46.4 MB/s 
     |████████████████████████████████| 1.2 MB 44.4 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 636 kB 46.5 MB/s 
     |████████████████████████████████| 6.5 MB 37.2 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 1.1 MB 61.6 MB/s 
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 53.7 MB/s 


In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 560.00 KiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-03-24 01:09:13--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=6M1o

In [ ]:
# !pip install mecab-python3
# !apt-get update
# !apt-get install g++ openjdk-8-jdk python-dev python3-dev
# !pip3 install JPype1-py3
# !pip3 install konlpy
# !JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

### 1.Import

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from konlpy.tag import Okt
from bertopic import BERTopic
import pandas as pd

### 2.Data read

In [ ]:
df = pd.read_excel('./20220311_review_data3.xlsx',0)
# df= df[df.columns[1:]]
doc = df.리뷰상세내용

In [ ]:
df.GROUP2.unique()[0]

'(미분류)'

In [ ]:
doc = list(doc)

In [ ]:
# def listToString(s):  
#     str1 = ""  
#     for ele in s:  
#         str1 += " " + ele.strip()  
#     return str1

In [ ]:
# doc = listToString(df.REVIEW)

In [ ]:
preprocessed_documents = []

for line in tqdm(doc):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 5218/5218 [00:00<00:00, 492005.45it/s]


In [ ]:
len(preprocessed_documents)

5216

### 3.Mecab과 SBERT를 이용한 Bertopic

In [ ]:
# class CustomTokenizer:
#     def __init__(self, tagger):
#         self.tagger = tagger
#     def __call__(self, sent):
#         sent = sent[:100000]
#         word_tokens = self.tagger.morphs(sent)
#         result = [word for word in word_tokens if len(word) > 1]
#         return result

In [ ]:
# custom_tokenizer = CustomTokenizer(Mecab())
# vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=20)

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:100000]
        word_tokens = self.tagger.nouns(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=20)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=30,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(preprocessed_documents)

In [ ]:
model.get_topic_info()

,Topic,Count,Name
0,-1,2324,-1_아이_항상_주문_아기
1,0,469,0_단계_먹이_분유_구매
2,1,133,1_매번_항상_주문_제품
3,2,118,2_아주_매일_항상_구매
4,3,113,3_매번_계속_명작_자주
5,4,110,4_구입_구매_계속_먹이
6,5,109,5_아이_아기_아주_매일
7,6,108,6__매일_아이_항상
8,7,101,7_구입_아주_주문_항상
9,8,100,8_제품_항상_자주_매번


In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_distribution(probs[0])

In [ ]:
for i in range(0, 22):
  print(i,'번째 토픽 :', model.get_topic(i))

In [ ]:
model2 = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=40,
                 top_n_words=20,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model2.fit_transform(preprocessed_documents)

In [ ]:
model2.visualize_topics()

In [ ]:
for i in range(0, 40):
  print(i,'번째 토픽 :', model2.get_topic(i))

0 번째 토픽 : [('는데', 0.03628560810416852), ('주문', 0.02931408588272827), ('네요', 0.027483931539044566), ('구매', 0.027013646192544285), ('아서', 0.026288423813190142), ('우유', 0.025977046119310024), ('으로', 0.025409179745938416), ('해서', 0.024490228256358833), ('기한', 0.024342440959895323), ('유통', 0.023337356652717453), ('어요', 0.021713543635142253), ('보다', 0.02059545029549675), ('너무', 0.019495225825511515), ('어서', 0.019174405147331302), ('해요', 0.01740301582208989), ('배송', 0.016625869911479213), ('', 1e-05), ('아요', 0.01544343534402069), ('아기', 0.013933893039282622), ('아이', 0.013435346271779636)]
1 번째 토픽 : [('맛있', 0.10418213421217089), ('어요', 0.04319276497107807), ('너무', 0.031905700145203046), ('네요', 0.025967142387659), ('아요', 0.01781256015064189), ('습니다', 0.017336567983132645), ('입니다', 0.016854262460634494), ('해요', 0.01449698841814726), ('우유', 0.012033956699104327), ('합니다', 0.008286289672282595), ('', 1e-05), ('포장', 0.007562792125830358), ('기한', 0.006824205670808262), ('어서', 0.0062761296182865705), 

In [ ]:
model3 = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=30,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model3.fit_transform(preprocessed_documents)

In [ ]:
model3.visualize_topics()

In [ ]:
for i in range(0, 40):
  print(i,'번째 토픽 :', model2.get_topic(i))

0 번째 토픽 : [('는데', 0.03628560810416852), ('주문', 0.02931408588272827), ('네요', 0.027483931539044566), ('구매', 0.027013646192544285), ('아서', 0.026288423813190142), ('우유', 0.025977046119310024), ('으로', 0.025409179745938416), ('해서', 0.024490228256358833), ('기한', 0.024342440959895323), ('유통', 0.023337356652717453), ('어요', 0.021713543635142253), ('보다', 0.02059545029549675), ('너무', 0.019495225825511515), ('어서', 0.019174405147331302), ('해요', 0.01740301582208989), ('배송', 0.016625869911479213), ('', 1e-05), ('아요', 0.01544343534402069), ('아기', 0.013933893039282622), ('아이', 0.013435346271779636)]
1 번째 토픽 : [('맛있', 0.10418213421217089), ('어요', 0.04319276497107807), ('너무', 0.031905700145203046), ('네요', 0.025967142387659), ('아요', 0.01781256015064189), ('습니다', 0.017336567983132645), ('입니다', 0.016854262460634494), ('해요', 0.01449698841814726), ('우유', 0.012033956699104327), ('합니다', 0.008286289672282595), ('', 1e-05), ('포장', 0.007562792125830358), ('기한', 0.006824205670808262), ('어서', 0.0062761296182865705), 